# Binomial Tree Calculator

## 20121229 JunPyo Park

In [11]:
import pandas as pd
import numpy as np

In [31]:
def option_pricing(option_type, s_0, k, u, d, n, r, T):
    row_data = []
    stock_data = []
    for i in range(n+1):
        value = s_0 * u **(n-i) * d ** i
        stock_data.append(value)
        if (option_type == 'Forward'):
            row_data.append(value-s_0*np.exp(r*T*n))
        elif (option_type.split()[1] == 'Call'):
            row_data.append(max(value-k, 0))
        elif (option_type.split()[1] == 'Put'):
            row_data.append(max(k-value, 0))
    
    final_value = (row_data, stock_data)
    
    try:
        if(option_type.split()[1] == 'Call'):
                code = 1 # Call -> code = 1
            elif(option_type.split()[1] == 'Put'):
                code = 2 # Put -> code = 2
        if (option_type.split()[0] == 'American'):
            result = American_backword_calculation(final_value, r, T, d, u, n, k, code)
        else:
            result = EUR_backword_calculation(final_value[0], r, T, d, u, n)
    except:
        result = EUR_backword_calculation(final_value[0], r, T, d, u, n)
    
    return result

In [32]:
def risk_neutral_pricing(data, r, T, d, u):
    p = (np.exp(r*T) - d) / (u - d)
    return (p * data[0] + (1-p) * data[1]) * np.exp(-r * T)

In [33]:
def EUR_backword_calculation(data, r, T, d, u, n):
    col = []
    for i in range(n+1):
        col.append('Period '+ str(i))
    tree_table = pd.DataFrame(columns = col)
    tree_table['Period '+ str(n)] = data
    for i in range(n):
        tree_table['Period '+ str(n-i-1)] = tree_table['Period '+ str(n-i)].rolling(window=2).apply(lambda x: risk_neutral_pricing(x,r,T,d,u))
    return tree_table

In [37]:
def American_backword_calculation(data, r, T, d, u, n, k, code):
    col = []
    for i in range(n+1):
        col.append('Period '+ str(i))
    
    tree_table = pd.DataFrame(columns = col)
    stock_table = pd.DataFrame(columns = col)
    
    tree_table['Period '+ str(n)] = data[0]
    stock_table['Period '+ str(n)] = data[1]
        
    for i in range(n):
        stock_table['Period '+ str(n-i-1)] = stock_table['Period '+ str(n-i)].rolling(window=2).apply(lambda x: x[0] / u)
        for j in range(n-i):
            x = [tree_table['Period '+ str(n-i)][i+j], tree_table['Period '+ str(n-i)][i+j+1]]
            rnp_price = risk_neutral_pricing(x,r,T,d,u)
            
            if (code==2): # Case of Put
                early_exe_value = k - stock_table['Period '+ str(n-i-1)][i+j+1]
            elif (code==1): # Case of Call
                early_exe_value = stock_table['Period '+ str(n-i-1)][i+j+1] - k
                
            tree_table['Period '+ str(n-i-1)][i+j+1] = max(rnp_price, early_exe_value)
            
    return tree_table

In [38]:
n_step = 4
upfactor = 1.1
downfactor = 0.9
r = 0.12 # interest rate
T = 0.25 # unit step time length
s_0 = 20 # initial stock price
k = 21  # strike price of that option

option_type = 'Forward' # Forward ignore the strike price k
result = option_pricing(option_type, s_0, k, upfactor, downfactor, n_step, r, T)
result

,Period 0,Period 1,Period 2,Period 3,Period 4
0,NaN,NaN,NaN,NaN,6.732063
1,NaN,NaN,NaN,4.736514,1.408063
2,NaN,NaN,2.963269,-0.103486,-2.947937
3,NaN,1.390909,-1.436731,-4.063486,-6.511937
4,5.925760e-15,-2.609091,-5.036731,-7.303486,-9.427937


In [39]:
option_type = 'European Call' # Forward ignore the strike price k
result = option_pricing(option_type, s_0, k, upfactor, downfactor, n_step, r, T)
result

,Period 0,Period 1,Period 2,Period 3,Period 4
0,NaN,NaN,NaN,NaN,8.282
1,NaN,NaN,NaN,6.240644,2.958
2,NaN,NaN,4.582139,1.872400,0.000
3,NaN,3.300424,1.185220,0.000000,0.000
4,2.342321,0.750238,0.000000,0.000000,0.000


In [40]:
option_type = 'European Put' # Forward ignore the strike price k
result = option_pricing(option_type, s_0, k, upfactor, downfactor, n_step, r, T)
result

,Period 0,Period 1,Period 2,Period 3,Period 4
0,NaN,NaN,NaN,NaN,0.000
1,NaN,NaN,NaN,0.000000,0.000
2,NaN,NaN,0.159194,0.471756,1.398
3,NaN,0.492979,1.162275,2.559356,4.962
4,0.96765,1.942793,3.577055,5.799356,7.878


In [41]:
option_type = 'American Call' # Forward ignore the strike price k
result = option_pricing(option_type, s_0, k, upfactor, downfactor, n_step, r, T)
result

,Period 0,Period 1,Period 2,Period 3,Period 4
0,NaN,NaN,NaN,NaN,8.282
1,NaN,NaN,NaN,6.240644,2.958
2,NaN,NaN,4.582139,1.872400,0.000
3,NaN,3.300424,1.185220,0.000000,0.000
4,2.342321,0.750238,0.000000,0.000000,0.000


In [42]:
option_type = 'American Put' # Forward ignore the strike price k
result = option_pricing(option_type, s_0, k, upfactor, downfactor, n_step, r, T)
result

,Period 0,Period 1,Period 2,Period 3,Period 4
0,NaN,NaN,NaN,NaN,0.000
1,NaN,NaN,NaN,0.000000,0.000
2,NaN,NaN,0.159194,0.471756,1.398
3,NaN,0.492979,1.162275,2.559356,4.962
4,0.96765,1.942793,3.577055,5.799356,7.878
